# Gas Station Price Analysis

In this Project we'll investigate the gas and diesel prices in Germany throughout the states, cities and gas stations. The dataset is provided by [Tankerkönig](https://tankerkoenig.de) under _Creative-Commons-Lizenz (CC BY 4.0)_.

## Importing and Accessing Data

The data provided by _Tankerkönig_ is a PostgreSQL Dump File. We need to first download a PostgreSQL Distribution.

For [MacOS](https://www.postgresql.org/download/macosx/), we can use [Postgres.app](http://postgresapp.com/), which is a simple, native macOS app that runs in the menubar without the need of an installer. There are several GUI client apps including [pgAdmin 4](https://www.pgadmin.org/) and [Postico](https://eggerapps.at/postico/).

For [Windows](https://www.postgresql.org/download/windows/), one can use the [Installer](https://www.enterprisedb.com/downloads/postgres-postgresql-downloads) which includes the PostgreSQL server, pgAdmin; a graphical tool for managing and developing your databases, and StackBuilder; a package manager that can be used to download and install additional PostgreSQL tools and drivers.

After installing the PostgreSQL, starting the server and and downloading the dump file, we need to restore the data from the dump. The following code can be used to restore the file. 

```
$ psql tanker < history.dump
```

Note that `tanker` is the name of our database.

### Data Description

There are two tables in the database:
1. `gas_station`: Information regarding gas stations, and their locations.
2. `gas_station_information_history`: Price information for the fuels E5, E10, and Diesel.

Detailed data description can be found [here](https://creativecommons.tankerkoenig.de).

### Connect to a PostgreSQL Database

In [1]:
# Import necessary modules and objects
from sqlalchemy import create_engine  # Connecting to database
from sqlalchemy import Table # Reflecting & viewing data
from sqlalchemy import MetaData # Reflecting & viewing data
from sqlalchemy import select # Selecting data

import pandas as pd # Feeding ResultProxy into pandas DataFrame.

In [3]:
# Load credentials
%run data/config

# Create an engine that connects to the database (with Connection URI)
engine = create_engine('postgresql+psycopg2://' + 
                       username + ':' + password + '@' +
                       host + ':' + port + '/' + database)

# Connect
connection = engine.connect()

In [19]:
# Table names
engine.table_names()

['gas_station', 'gas_station_information_history']

In [20]:
# Initialize metadata
metadata = MetaData()

# Reflect tables from the engine
gas_station = Table('gas_station', metadata, autoload=True, autoload_with=engine)
gas_price = Table('gas_station_information_history', metadata, autoload=True, autoload_with=engine)

In [21]:
gas_station.columns.keys()

['id',
 'version',
 'version_time',
 'name',
 'brand',
 'street',
 'house_number',
 'post_code',
 'place',
 'public_holiday_identifier',
 'lat',
 'lng',
 'price_in_import',
 'price_changed',
 'open_ts',
 'ot_json',
 'station_in_import',
 'first_active']

In [22]:
gas_price.columns.keys()

['id', 'stid', 'e5', 'e10', 'diesel', 'date', 'changed']

### Functions

In [ ]:
# Function to extract table to a pandas DataFrame
def table_to_pandas(tablename, db_engine):
    '''Extract table to a pandas DataFrame.'''

    query = "SELECT * FROM {}".format(tablename)
    return pd.read_sql(query, db_engine)

# # Extract the table into a pandas DataFrame
# table_to_pandas('table1', db_engine)

# # Extract the customer table into a pandas DataFrame
# table_to_pandas('table2', db_engine)

### Selecting Data

In [25]:
# Build select statement
stmt = '''
SELECT * 
FROM gas_station_information_history
LIMIT 5
'''

# Execute the statement (Result Proxy)
results_proxy = connection.execute(stmt)

# Fetch the results (Result Set)
results = results_proxy.fetchall()

# Print the first five results
results[:1]

[(154134533, UUID('dce91c1f-80ea-4a60-be4c-c94b46140a07'), 1339, 1319, 1249, datetime.datetime(2019, 1, 1, 19, 8, 6, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=60, name=None)), 5)]

In [26]:
# Create a DataFrame from the results
df = pd.DataFrame(results)

# Set column names
df.columns = results[0].keys()

# Print the head of the Dataframe
df.head()

,id,stid,e5,e10,diesel,date,changed
0,154134533,dce91c1f-80ea-4a60-be4c-c94b46140a07,1339,1319,1249,2019-01-01 19:08:06+01:00,5
1,154134534,482ed2fa-8f25-48d7-8319-62d78bb80dad,1369,1349,1209,2019-01-01 19:08:06+01:00,1
2,154134535,f96b0d2b-c33b-4d8e-a08b-6a9e2f3c657a,1419,1399,1339,2019-01-01 19:08:06+01:00,21
3,154134536,298b75ef-7296-4ebe-ab06-536f4782032b,1309,1289,1149,2019-01-01 19:08:06+01:00,21
4,154134537,6a8c91b0-7b3f-4c99-a2c9-787aa0bcd829,1339,0,1189,2019-01-01 19:08:06+01:00,4


In [30]:
# Build select statement
stmt = '''
SELECT * 
FROM gas_station
LIMIT 5
'''

# Create the same DataFrame with less code
df = pd.read_sql(stmt, engine)

# Print the head of the Dataframe
df.head()

,id,version,version_time,name,brand,street,house_number,post_code,place,public_holiday_identifier,lat,lng,price_in_import,price_changed,open_ts,ot_json,station_in_import,first_active
0,00060162-0001-4444-8888-acdc00000001,2,2019-06-29 00:00:18.081,Tankstelle MGS,,Justus-Liebig-Str.,96,95447,Bayreuth,None,49.938751,11.559593,2019-06-30 15:10:06+00:00,2019-06-30 15:04:06+00:00,1,{},2019-06-29 22:00:35+00:00,2017-08-25 23:30:06+02:00
1,00060251-0012-4444-8888-acdcffffffff,0,2019-06-29 00:00:09.807,Raiffeisen Bezug+Absatz eG,AVIA,Kempter Str.,11,87487,Wiggensbach,None,47.747704,10.239517,2019-06-30 15:10:06+00:00,2019-06-19 14:03:06+00:00,1,{},2019-06-29 22:00:35+00:00,2014-03-18 16:45:31+01:00
2,00099999-8026-4444-8888-acdc00008026,0,2019-06-29 00:00:15.952,ch-tank,ch-tank,Freckenhorster Straße,39,48231,Warendorf,None,51.949173,7.987864,2019-06-30 15:10:06+00:00,2019-06-30 14:52:14+00:00,1,{},2019-06-29 22:00:35+00:00,2014-05-27 00:00:01+02:00
3,005056ba-7cb6-1ed2-bceb-5df56c7ecd1b,1896,2019-06-27 22:55:10.785,famila Tankstelle,FAMILA,Rettiner Weg,77,23730,Neustadt / Holstein,None,54.103570,10.831820,2019-06-30 15:10:06+00:00,2019-06-30 14:25:06+00:00,-1561971600,"{""openingTimes"":[{""applicable_days"":63,""period...",2019-06-29 22:00:35+00:00,2014-03-18 16:45:31+01:00
4,005056ba-7cb6-1ed2-bceb-7ac58dfa4d27,1896,2019-06-27 22:55:11.652,star Tankstelle,STAR,Celler Straße,55,29303,Bergen,None,52.800090,9.969620,2019-06-30 15:10:06+00:00,2019-06-30 14:59:06+00:00,1,{},2019-06-29 22:00:35+00:00,2014-03-18 16:45:31+01:00


In [35]:
stmt = '''
SELECT id, brand
FROM gas_station
WHERE place = 'Düsseldorf'
'''

# Create a DataFrame from the results
df = pd.read_sql(stmt, engine)

# Print the head of the Dataframe
df.head()

,id,brand
0,005056ba-7cb6-1ed2-bceb-7e82e4910d2a,STAR
1,005056ba-7cb6-1ed2-bceb-7ef561844d2a,STAR
2,005056ba-7cb6-1ed2-bceb-80c585ca6d2b,STAR
3,09ee977e-c04e-4db1-98bf-b507b123423c,SVG Düsseldorf
4,1241bc5a-5571-4cee-bce0-d0ab82000d8c,ARAL


In [39]:
# Select two gas stations for initial investigation
stmt = '''
SELECT *
FROM gas_station_information_history
JOIN gas_station
    ON gas_station.id = gas_station_information_history.stid
WHERE (gas_station.id = '005056ba-7cb6-1ed2-bceb-7e82e4910d2a'
    OR gas_station.id = '1241bc5a-5571-4cee-bce0-d0ab82000d8c')
    AND gas_station_information_history.date > '2016-01-01 00:00:00+01'
'''

# Create a DataFrame from the results
df = pd.read_sql(stmt, engine)

# Print the head of the Dataframe
df.head()

,id,stid,e5,e10,diesel,date,changed,id,version,version_time,...,place,public_holiday_identifier,lat,lng,price_in_import,price_changed,open_ts,ot_json,station_in_import,first_active
0,2326,1241bc5a-5571-4cee-bce0-d0ab82000d8c,1309,1289,1109,2016-10-15 15:50:07+02:00,63,1241bc5a-5571-4cee-bce0-d0ab82000d8c,4,2016-02-01 10:29:44.000,...,Düsseldorf,None,51.20101,6.763259,2019-06-30 15:10:06+00:00,2019-06-30 14:18:06+00:00,1,{},2019-06-29 22:00:35+00:00,2014-03-18 15:45:31+00:00
1,6910,005056ba-7cb6-1ed2-bceb-7e82e4910d2a,1269,1249,1069,2016-10-15 15:50:07+02:00,63,005056ba-7cb6-1ed2-bceb-7e82e4910d2a,1896,2019-06-27 22:55:09.758,...,Düsseldorf,None,51.23210,6.723630,2019-06-30 15:10:06+00:00,2019-06-30 15:03:06+00:00,1561924800,"{""openingTimes"":[{""applicable_days"":15,""period...",2019-06-29 22:00:35+00:00,2014-03-18 15:45:31+00:00
2,15924,1241bc5a-5571-4cee-bce0-d0ab82000d8c,1439,1419,1189,2016-05-01 22:05:02+02:00,21,1241bc5a-5571-4cee-bce0-d0ab82000d8c,4,2016-02-01 10:29:44.000,...,Düsseldorf,None,51.20101,6.763259,2019-06-30 15:10:06+00:00,2019-06-30 14:18:06+00:00,1,{},2019-06-29 22:00:35+00:00,2014-03-18 15:45:31+00:00
3,2337,1241bc5a-5571-4cee-bce0-d0ab82000d8c,1289,1269,1039,2016-05-01 21:26:06+02:00,63,1241bc5a-5571-4cee-bce0-d0ab82000d8c,4,2016-02-01 10:29:44.000,...,Düsseldorf,None,51.20101,6.763259,2019-06-30 15:10:06+00:00,2019-06-30 14:18:06+00:00,1,{},2019-06-29 22:00:35+00:00,2014-03-18 15:45:31+00:00
4,6487,005056ba-7cb6-1ed2-bceb-7e82e4910d2a,1249,1229,999,2016-05-01 21:26:06+02:00,63,005056ba-7cb6-1ed2-bceb-7e82e4910d2a,1896,2019-06-27 22:55:09.758,...,Düsseldorf,None,51.23210,6.723630,2019-06-30 15:10:06+00:00,2019-06-30 15:03:06+00:00,1561924800,"{""openingTimes"":[{""applicable_days"":15,""period...",2019-06-29 22:00:35+00:00,2014-03-18 15:45:31+00:00


In [40]:
df.shape

(26830, 25)

## Saving Data

In [41]:
df.to_pickle('data/two-stations.pkl')

In [ ]:
connection.close()